# Solution Guide

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
movies = pd.read_csv('../data/tmdb_5000_movies.csv')
credits = pd.read_csv('../data/tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
movies.shape

In [ ]:
credits.head()

In [ ]:
credits.shape

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L 

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
def convert3(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count < 3:
            L.append(i['name'])
        count +=1  
    return L    

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L   

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.overview[0]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [ ]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [ ]:
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x :" ".join(x))

In [ ]:
new_df['tags'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000 ,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vectors).shape

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
sorted(list(enumerate(similarity[0])),reverse =True , key = lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0] ##fetching the movie index
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate( distances)),reverse =True , key = lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

Source:

-https://www.kaggle.com/code/kachalisarma20/tmdb-movie-recommender-system